# Task C: Masks Sold by Etsy
1. Go to the website below and estimate how many masks are sold daily and daily sales (in $) generated by masks during a 2 week period (feel free to be creative here, note we asking for an accurate estimate not exact number)
> https://www.etsy.com/


In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import datetime
import time
import os

import selenium
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

# 1 Extract Data from Etsy

## Method I: Avg Price * Delta Sales  * currency for each facemask shop
Automatically extract via
> Mask.py

export data to
> 2020-06-xx.csv 

In [6]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}
    return BeautifulSoup(requests.get(url=url, headers=headers).content, "html.parser")
    


def get_shop_list(api = "dy8fex63879fnclp7em1j90i"):
    shop_list = []
    
        
    #"limit" attribute to expand records from default 25 to 100 shops
    #facemask shop number is about around 700
    for i in range(8):
        offset = i * 100
        site = 'https://openapi.etsy.com/v2/shops?limit=100&offset=%f&shop_name=face*mask&api_key=%s'%(offset,api )
        webpage = requests.get(site)
        shop_json = json.loads(webpage.text)
    
        for s in shop_json['results']:
            shop_list.append((s['shop_id'], s['shop_name'],
                              s["currency_code"],s['listing_active_count']))
    return shop_list

## from product - (stock quantity *price)
def get_product_info(shoplist, api = 'dy8fex63879fnclp7em1j90i'):
    shoplist_ = shoplist
    shop_output = {}
    time.sleep(1)
    
    for i,shop in enumerate(shoplist_):
        if i%5 ==0: 
            time.sleep(0.2) #api limit 5 times per second
            
        shopId, title, currency, listing_active_count = shop[0],shop[1],shop[2], shop[3]
        shop_output[shopId]={}
        shop_output[shopId]['title'] = title
        shop_output[shopId]['currency'] = currency
        
        #extract active listing products from each shop
        #just consider first 100 products
        url = 'https://openapi.etsy.com/v2/shops/%f/listings/active?limit=100&api_key=%s'%(shopId,api)
        listing = requests.get(url)
        listing_json = json.loads(listing.text)
        
        #get listing product info
        shop_output[shopId]['listing'] = {}
        price_favorer = 0
        total_favorer = 0
        p = 0
        for product in listing_json['results']:
            listing_id = product['listing_id']
            shop_output[shopId]['listing'][listing_id] = {}
            price = float(product['price'].replace(",",''))
            shop_output[shopId]['listing'][listing_id]['price'] = price
            shop_output[shopId]['listing'][listing_id]['num_favorers'] = product['num_favorers']
            shop_output[shopId]['listing'][listing_id]['quantity'] = product['quantity']
            price_favorer += price * product['num_favorers']
            total_favorer += product['num_favorers']
            p += price
        
        #Avg Price
        try:
            avg_price =  price_favorer / total_favorer
        except ZeroDivisionError:
            try:
                avg_price = p / listing_active_count
            except ZeroDivisionError:
                avg_price = 0
        shop_output[shopId]['avg_price'] = avg_price
        #shop average price weighted by number of product favorer
        
        
        #get shop historical sales
        shop_web = 'https://www.etsy.com/shop/%s?ref=ss_profile'%shop_output[shopId]['title']
        soup = get_html(shop_web)
        
        try:
            sale = soup.find('span', attrs = {'class':'shop-sales hide-border no-wrap'})
        except KeyError:
            print(shop_web)
            continue

        try:
            sales = float(sale.text.replace(' Sales','').replace(',','').replace(' Sale',''))
            shop_output[shopId]['sale'] = sales 
        except AttributeError:
            shop_output[shopId]['sale'] = 0
        
    return shop_output

  

In [154]:
shop_list = get_shop_list(api = 'dy8fex63879fnclp7em1j90i')


In [186]:
output = get_product_info(shop_list)

In [187]:
d = datetime.datetime.now().strftime('%Y-%m-%d')
with open('%s.json'%d, 'w') as f:
    json.dump(output, f)

In [188]:
output

{7570370: {'title': 'FaceMasksByGen',
  'currency': 'USD',
  'listing': {820491993: {'price': 10.0, 'num_favorers': 0, 'quantity': 19},
   820486559: {'price': 10.0, 'num_favorers': 0, 'quantity': 10},
   806574222: {'price': 10.0, 'num_favorers': 0, 'quantity': 10},
   800062132: {'price': 10.0, 'num_favorers': 1, 'quantity': 40},
   803406912: {'price': 10.0, 'num_favorers': 0, 'quantity': 6},
   806131674: {'price': 10.0, 'num_favorers': 0, 'quantity': 112},
   799853762: {'price': 31.0, 'num_favorers': 1, 'quantity': 48},
   804662092: {'price': 10.0, 'num_favorers': 0, 'quantity': 30},
   804589368: {'price': 10.0, 'num_favorers': 0, 'quantity': 10},
   804587514: {'price': 10.0, 'num_favorers': 0, 'quantity': 15},
   800066880: {'price': 10.0, 'num_favorers': 0, 'quantity': 13},
   803401254: {'price': 10.0, 'num_favorers': 0, 'quantity': 22},
   817287763: {'price': 10.0, 'num_favorers': 0, 'quantity': 16},
   803403140: {'price': 10.0, 'num_favorers': 0, 'quantity': 8},
   8033

* First number 7570370 (7 digits) is ShopId, number with 9 digits is ListingId

In [148]:
d = datetime.datetime.now().strftime('%Y-%m-%d')
d

'2020-06-07'

# 2 Extract Currency 
Automatically extract via
> currency.py

export data to
> currency.csv 

In [27]:
pd.read_json('2020-06-07.json').T.currency.unique()

array(['USD', 'GBP', 'HKD', 'CAD', 'EUR', 'SEK', 'CHF', 'NZD', 'AUD'],
      dtype=object)

In [28]:
c_list = list(pd.read_json('2020-06-07.json').T.currency.unique())
c_list

['USD', 'GBP', 'HKD', 'CAD', 'EUR', 'SEK', 'CHF', 'NZD', 'AUD']

In [33]:
currency = []
for c in c_list:
    link = 'https://widget-yahoo.ofx.com/?base=%s&term=USD'%c
    opts = Options()
    opts.headless = True
    #opts.sandbox = False
    chromepath = '/usr/local/bin/chromedriver'
    browser = Chrome(chromepath, options=opts)
    browser.implicitly_wait(1)
    
    browser.get(link)
    time.sleep(2)
    value = browser.find_element_by_class_name('title-to-price').text
    browser.close()
    currency.append(value)
    
print(currency)

['1.0000', '1.2509', '0.1290', '0.7356', '1.1249', '0.1071', '1.0504', '0.6432', '0.6844']


In [39]:
d = datetime.datetime.now().strftime('%Y-%m-%d')
c_list.append('Date')
currency.append(d)

with open('currency.csv','w') as f:
    f.write(",".join(c_list))
    f.write('\n')
    f.write(",".join(currency))
    f.write('\n')
f.close()
    

In [ ]:
# Daily extracting use currency.py

# 3 Preprocess the data

### 3.1 Sample and data structure Visualize: Daily sales for 2020-6-8

In [8]:
d1 = pd.read_json('2020-06-23.json').T

d1['ShopId'] = d1.index
d1.reset_index(drop = True,inplace=True)
d1.rename(columns = {'sale':'d1_sale'}, inplace = True)
d1.head()

,avg_price,currency,listing,d1_sale,title,ShopId
0,14.75,USD,"{'786388554': {'price': 14.75, 'num_favorers':...",924,FaceMasksbySusie,7437460
1,19.3509,USD,"{'814029881': {'price': 10.0, 'num_favorers': ...",63,FaceMasksByGen,7570370
2,8.15989,USD,"{'496821981': {'price': 30.0, 'num_favorers': ...",246,surfponchos,8313110
3,9.95,USD,"{'790806428': {'price': 9.95, 'num_favorers': ...",79,FaceMasksByRachel,11067886
4,8.4488,USD,"{'798054272': {'price': 3.0, 'num_favorers': 5...",937,FaceMaskParlor,11287272


In [9]:
d2 = pd.read_json('2020-06-24.json').T

d2['ShopId'] = d2.index
d2.reset_index(drop = True,inplace=True)
d2.rename(columns = {'sale':'d2_sale'}, inplace = True)
d2.head()

,avg_price,currency,listing,d2_sale,title,ShopId
0,19.1897,USD,"{'817287763': {'price': 10.0, 'num_favorers': ...",67,FaceMasksByGen,7570370
1,0,USD,{},45569,FaceMasksThatFit,7875946
2,8.42416,USD,"{'217068327': {'price': 5.0, 'num_favorers': 4...",247,surfponchos,8313110
3,9.95,USD,"{'824876031': {'price': 9.95, 'num_favorers': ...",85,FaceMasksByRachel,11067886
4,8.45279,USD,"{'808424817': {'price': 9.95, 'num_favorers': ...",952,FaceMaskParlor,11287272


In [10]:
#merge two day sale
d_sale = d2.merge(d1[['d1_sale','ShopId']],on = "ShopId",how = 'inner')
d_sale['daily_sale'] = d_sale['d2_sale'] - d_sale['d1_sale']
d_sale.shape

(732, 8)

In [15]:
pd.read_json('2020-06-23.json').T.currency.unique()

array(['USD', 'GBP', 'HKD', 'CAD', 'EUR', 'SEK', 'TRY', 'CHF', 'AUD',
       'NZD'], dtype=object)

In [20]:
#merge currency
cur_exchange = pd.read_csv('currency.csv')
cur_exchange.tail()

,USD,GBP,HKD,CAD,EUR,SEK,CHF,NZD,AUD,Date
8,1.0,1.2415,0.1297,0.7384,1.1233,0.1061,1.0552,0.6441,0.6869,2020-06-20
9,1.0,1.2350,0.1290,0.7345,1.1185,0.1061,1.0507,0.6402,0.6825,2020-06-21
10,1.0,1.2489,0.1290,0.7400,1.1273,0.1069,1.0564,0.6490,0.6926,2020-06-22
11,1.0,1.2532,0.1290,0.7382,1.1326,0.1081,1.0596,0.6487,0.6956,2020-06-23
12,1.0,1.2412,0.1290,0.7331,1.1252,0.1069,1.0548,0.6408,0.6861,2020-06-24


In [17]:
d_sale['cur_value'] = 0
for i in range(d_sale.shape[0]):
    cur = d_sale.loc[i,'currency']
    if cur == 'TRY':
        cur_value = 0.1460
    else:
        cur_value = cur_exchange[cur_exchange['Date'] == '2020-06-12'][cur].values[0]
    d_sale.loc[i,'cur_value'] = cur_value
    
d_sale['revenue'] = d_sale['avg_price'] * d_sale['daily_sale'] * d_sale['cur_value']
d_sale.head()

,avg_price,currency,listing,d2_sale,title,ShopId,d1_sale,daily_sale,cur_value,revenue
0,19.1897,USD,"{'817287763': {'price': 10.0, 'num_favorers': ...",67,FaceMasksByGen,7570370,63,4,1.0,76.7586
1,8.42416,USD,"{'217068327': {'price': 5.0, 'num_favorers': 4...",247,surfponchos,8313110,246,1,1.0,8.42416
2,9.95,USD,"{'824876031': {'price': 9.95, 'num_favorers': ...",85,FaceMasksByRachel,11067886,79,6,1.0,59.7
3,8.45279,USD,"{'808424817': {'price': 9.95, 'num_favorers': ...",952,FaceMaskParlor,11287272,937,15,1.0,126.792
4,17.3713,USD,"{'827524425': {'price': 17.99, 'num_favorers':...",849,FaceMaskQueen,11597812,829,20,1.0,347.426


In [124]:
d_sale = d_sale[['avg_price','currency','cur_value','daily_sale','title','ShopId','revenue']]
d_sale.head()

,avg_price,currency,cur_value,daily_sale,title,ShopId,revenue
0,9.95,USD,1.0,3,FaceMasksByRachel,11067886,29.85
1,8.51443,USD,1.0,19,FaceMaskParlor,11287272,161.774
2,17.5453,USD,1.0,15,FaceMaskQueen,11597812,263.18
3,14.9071,USD,1.0,0,FaceMasksbyViv,14625203,0
4,18.3413,USD,1.0,2,FaceMaskHub,20279847,36.6827


In [125]:
d_sale.shape

(690, 7)

In [123]:
d_sale[d_sale.cur_value!=1.0].head()

,avg_price,currency,cur_value,daily_sale,title,ShopId,revenue
11,5.35304,GBP,1.2509,375,FaceMasksLondon,22681525,2511.04
13,76.4058,HKD,0.1290,5,FaceMaskCottonFabric,22730471,49.2818
15,14.7457,EUR,1.1249,1,FaceMaskUSafe,22765057,16.5874
32,5.06835,GBP,1.2509,3,FaceMask4UShop,22903762,19.02
35,7.09033,GBP,1.2509,413,FaceMaskYard,22923315,3663.02


In [126]:
d_sale.head()

,avg_price,currency,cur_value,daily_sale,title,ShopId,revenue
0,9.95,USD,1.0,3,FaceMasksByRachel,11067886,29.85
1,8.51443,USD,1.0,19,FaceMaskParlor,11287272,161.774
2,17.5453,USD,1.0,15,FaceMaskQueen,11597812,263.18
3,14.9071,USD,1.0,0,FaceMasksbyViv,14625203,0
4,18.3413,USD,1.0,2,FaceMaskHub,20279847,36.6827


In [127]:
d_sale.daily_sale.sum()

2487.0

In [128]:
d_sale.revenue.sum()

25040.02317682542

### 3.2 CSV daily sales for all files

In [5]:
a = os.listdir()
dirlist = []
for i in a:
    if i.startswith('2020-') == True:
        dirlist.append(i)
dirlist.sort()
dirlist

['2020-06-07.json',
 '2020-06-08.json',
 '2020-06-10.json',
 '2020-06-11.json',
 '2020-06-12.json',
 '2020-06-14.json',
 '2020-06-15.json',
 '2020-06-16.json',
 '2020-06-17.json',
 '2020-06-18.json',
 '2020-06-19.json',
 '2020-06-20.json',
 '2020-06-21.json',
 '2020-06-22.json',
 '2020-06-23.json',
 '2020-06-24.json']

In [19]:
daily_item_sold = []
daily_revenue = []
date_list = []
for i,file in enumerate(dirlist[:-1]):
    #day1
    d1 = pd.read_json(file).T
    d1['ShopId'] = d1.index
    d1.reset_index(drop = True,inplace=True)
    d1.rename(columns = {'sale':'d1_sale'}, inplace = True)
    
    #day2
    d2 = pd.read_json(dirlist[i+1]).T
    d2['ShopId'] = d2.index
    d2.reset_index(drop = True,inplace=True)
    d2.rename(columns = {'sale':'d2_sale'}, inplace = True)
    
    #merge two day sale
    d_sale = d2.merge(d1[['d1_sale','ShopId']],on = "ShopId",how = 'inner')
        #process for skipped day
    if file in ['2020-06-08.json', '2020-06-12.json']:
        d_sale['daily_sale'] = (d_sale['d2_sale'] - d_sale['d1_sale'])/2
    else:
        d_sale['daily_sale'] = d_sale['d2_sale'] - d_sale['d1_sale']
    
    #merge currency
    cur_exchange = pd.read_csv('currency.csv')
    d_sale['cur_value'] = 0
    date = dirlist[i+1][:10]
    if date not in cur_exchange['Date'].values:
        date = cur_exchange['Date'].values[0]
    for i in range(d_sale.shape[0]):
        cur = d_sale.loc[i,'currency']
        if cur == 'TRY':
            cur_value = 0.1460 # 6-24 A New currency used for shop is TRY
        else:
            cur_value = cur_exchange[cur_exchange['Date'] == date][cur].values[0]
        d_sale.loc[i,'cur_value'] = cur_value
        
    #Total Daily Revenue
    d_sale['revenue'] = d_sale['avg_price'] * d_sale['daily_sale'] * d_sale['cur_value']
    item_sold = d_sale.daily_sale.sum()
    revenue = d_sale.revenue.sum()
    if file in ['2020-06-08.json','2020-06-12.json']:
        daily_item_sold += [item_sold]*2
        daily_revenue += [revenue]*2
        if file == '2020-06-08.json':
            date_list += ['2020-06-09','2020-06-10']
        else:
            date_list += ['2020-06-13','2020-06-14']
    else:
        date_list.append(date)
        daily_item_sold.append(item_sold)
        daily_revenue.append(revenue)

tmp = {'Item_sold':daily_item_sold,
       'Revenue':daily_revenue,
       'Date': date_list}   
#print(len(date_list),len(daily_item_sold))
df = pd.DataFrame(tmp)
df.to_csv('Etsy_Mask_sale.csv',index=False)
        
        
        
    

In [ ]:
df.to_csv('Etsy_Mask_sale.csv',index=False)

## Method II: In stock Quantity Delta * price, for each listings

In [2]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}
    return BeautifulSoup(requests.get(url=url, headers=headers).content, "html.parser")
    #return BeautifulSoup(requests.get(url=url).content, "html.parser")



#p = {'query':'mask',
#             'page':[x for x in range(1,251)]}
#site = 'https://www.etsy.com/c/bath-and-beauty/personal-care/face-masks-and-coverings?ref=pagination&page=%f'%1
#webpage = get_html(site)
#print(webpage)

In [87]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}
    return BeautifulSoup(requests.get(url=url, headers=headers).content, "html.parser")
    #return BeautifulSoup(requests.get(url=url).content, "html.parser")


def get_href_list():
    href_list = []
    for page in range(1,251): 
        site = 'https://www.etsy.com/c/bath-and-beauty/personal-care/face-masks-and-coverings?ref=pagination&page=%f'%page
        webpage = get_html(site)
        chart = webpage.find_all('a', attrs={'class':' display-inline-block listing-link '})
        href_list =href_list + [chart[x]['href'] for x in range(len(chart))]
    return href_list

## from product - (stock quantity *price)
def get_product_info(url_list):
    Pdt = {}
    
    for i,url in enumerate(url_list):
        if i%6 ==0: 
            time.sleep(1) #api limit 8 times per second
        soup = get_html(url)
        
        try:
            product = soup.find_all('script', attrs = {'type':'text/javascript'})
        except KeyError:
            print(url)
            continue

        try:
            product_json = json.loads(product[3].text.replace('\n','').replace(' ','').split('window.dataLayer=')[1][:-1])[0]
        except json.decoder.JSONDecodeError:
            print(product[3])
            continue
        try:
            product_id = product_json['granify_product']['id']
            if product_id  not in Pdt.keys():
                Pdt[product_id] = {}
                Pdt[product_id]['price'] = product_json['granify_product']['price']
                Pdt[product_id]['in_stock'] = product_json['granify_product']['in_stock']
                Pdt[product_id]['title'] = product_json['granify_product']['title']
                Pdt[product_id]['Date'] = datetime.datetime.now().strftime("%Y-%m-%d")
        except KeyError:
            print(url)
            continue
        
    return Pdt

  

In [70]:
urls = get_href_list()
len(urls)

12000

In [72]:
len(urls)

12000

In [97]:
Pdt = {}
h = urls

for i,url in enumerate(h):
    if i%6 ==0: 
        time.sleep(1) #api limit 8 times per second
    soup = get_html(url)

    try:
        product = soup.find_all('script', attrs = {'type':'text/javascript'})
    except KeyError:
        print(url)
        continue
    
    try:
        product_json = json.loads(product[3].text.replace('\n','').replace(' ','').split('window.dataLayer=')[1][:-1])[0]
    except json.decoder.JSONDecodeError:
        print(product[3])
        continue
    try:
        product_id = product_json['granify_product']['id']
        if product_id  not in Pdt.keys():
            Pdt[product_id] = {}
            Pdt[product_id]['price'] = product_json['granify_product']['price']
            Pdt[product_id]['in_stock'] = product_json['granify_product']['in_stock']
            Pdt[product_id]['title'] = product_json['granify_product']['title']
            Pdt[product_id]['currency'] = product_json['currency']
            Pdt[product_id]['Date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    except KeyError:
        print(url)
        continue
    

https://www.etsy.com/listing/288440675/red-splatter-adult-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-1-14&bes=1
https://www.etsy.com/listing/805015605/reusable-face-mask-washable-w-no-rust?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-1-26&pro=1&col=1
https://www.etsy.com/listing/661004647/3-pack-bandana-face-mask-multi?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-1-30&frs=1&bes=1
https://www.etsy.com/listing/691735689/fabric-face-mask-filter-pocket-made-in?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-1-41&frs=1&bes=1
https://www.etsy.com/listing/787091362/indigo-linen-face-mask-whimsical-face?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-1-47
https://www.etsy.com/listing/780270094/viking-face-shield-mask-cover-fully?ga_order=most

https://www.etsy.com/listing/775538046/face-mask-cover-reusable-adult-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-10-10&pro=1
https://www.etsy.com/listing/806393494/beer-face-mask-face-cover-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-10-15&frs=1
https://www.etsy.com/listing/786511428/fabric-face-mask-adult-large-washable?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-10-16&bes=1
https://www.etsy.com/listing/811343709/navyfuchsia-patterneddesigned-bandana?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-10-34
https://www.etsy.com/listing/817417759/i-cant-breathe-face-mask-george-floyd?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-11-17
https://www.etsy.com/listing/802946984/bendable-metal-nose-bridge-aluminum-wire?ga_order=most_re

https://www.etsy.com/listing/810658983/african-kente-face-mask-with-filter?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-18-47
https://www.etsy.com/listing/806396420/4th-of-july-democrat-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-19-22
https://www.etsy.com/listing/795488937/black-face-mask-unisex-breathable?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-19-43&bes=1&col=1
https://www.etsy.com/listing/819214599/thin-and-breathable-eco-friendly?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-20-11&frs=1
https://www.etsy.com/listing/805959622/black-lives-matter-face-mask-unisex-face?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-20-17&pro=1&frs=1
https://www.etsy.com/listing/820287679/womens-face-mask-nose-wire-pocket-for?ga_order=most_rele

https://www.etsy.com/listing/802719423/joker-printed-face-mask-replaceable?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-27-29
https://www.etsy.com/listing/779312852/3-layer-cotton-face-mask-adults-reusable?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-27-44&frs=1&bes=1&col=1
https://www.etsy.com/listing/802602862/designer-face-mask-washable-reusable-w?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-27-47
https://www.etsy.com/listing/817773349/ferrari-logo-custom-personalized-face?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-28-10&cns=1
https://www.etsy.com/listing/820285813/50pack-color-face-mask-silky-modal-blank?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-28-11&frs=1
https://www.etsy.com/listing/783018450/disney-stitch-face-mask?ga_order=mos

https://www.etsy.com/listing/788829922/100-cotton-green-bay-packers-face?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-36-18&bes=1
https://www.etsy.com/listing/811274253/colorado-face-dust-shield-mask-with?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-36-33&frs=1
https://www.etsy.com/listing/778217862/reusable-cotton-face-mask-adjustable?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-36-38&frs=1&bes=1
https://www.etsy.com/listing/787649630/usa-made-cotton-face-mask-3-layered-face?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-37-1&pro=1&col=1
https://www.etsy.com/listing/782975712/martial-arts-kick-your-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-37-4
https://www.etsy.com/listing/806395108/handmade-face-mask-100-cotton-glasses?ga_order

...

https://www.etsy.com/listing/789674052/3-layer-fitted-filter-pocket-mini?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-209-9&bes=1
https://www.etsy.com/listing/778914770/2-pack-washable-face-mask-w-filter?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-209-14&bes=1
https://www.etsy.com/listing/795643368/adult-reusable-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-209-19&col=1
https://www.etsy.com/listing/806354108/catrina-3-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-209-21
https://www.etsy.com/listing/796164630/music-lover-face-mask-gift-for-music?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-209-27&frs=1
https://www.etsy.com/listing/786531522/lamborghini-aventador-face-mask-for?ga_order=most_relevant&ga_search_type=all&ga_v

https://www.etsy.com/listing/806350796/black-lives-matter-blm-cotton-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-219-1&frs=1
https://www.etsy.com/listing/804122361/bambi-face-mask-lined-w-cotton-canvas?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-219-13&frs=1
https://www.etsy.com/listing/819768465/face-mask-with-filter-pocket-and-nose?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-219-28&col=1
https://www.etsy.com/listing/786096298/haitian-face-covering-washable-and?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-219-29&frs=1
https://www.etsy.com/listing/794096768/handmade-face-masks-uk-charity-donation?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-219-30&bes=1
https://www.etsy.com/listing/779769782/surgical-face-mask-ear-saver-strain?

https://www.etsy.com/listing/797606464/face-masksadulthello?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-228-11&bes=1
https://www.etsy.com/listing/820230993/original-design-design-cotton-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-228-12&frs=1
https://www.etsy.com/listing/820234395/original-design-design-cotton-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-228-24&frs=1
https://www.etsy.com/listing/817179849/black-lives-matter-logo2-100-organic?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-228-33&pro=1&col=1
https://www.etsy.com/listing/799039655/washable-reusable-face-mask-with-filters?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-228-35&bes=1
https://www.etsy.com/listing/794906883/cotton-reusable-facemask-with-filter?ga_o

https://www.etsy.com/listing/792111720/bicycle-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-237-32
https://www.etsy.com/listing/776938990/quick-delivery-100-adjustable-eco?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-237-33&pro=1&col=1
https://www.etsy.com/listing/817954891/philadelphia-eagles-sportsmask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-237-36
https://www.etsy.com/listing/804059051/diamond-studded-print-two-ply-mask-made?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-237-42&col=1
https://www.etsy.com/listing/806345462/black-lives-matter-blm-black-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-238-6&frs=1
https://www.etsy.com/listing/797311431/womens-face-mask-100-cotton-washable?ga_order=most_relevant&ga_search_t

https://www.etsy.com/listing/804418317/cocoretroglam-3-layer-cotton-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-245-46&frs=1
https://www.etsy.com/listing/795811470/dinosaurs-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-246-2&cns=1&col=1
https://www.etsy.com/listing/802414695/sponge-bob-fabric-face-mask-washable?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-246-4&frs=1&cns=1
https://www.etsy.com/listing/807375343/yankees-face-mask-reusable-face-mask?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-246-7&bes=1
https://www.etsy.com/listing/804597330/white-sox-face-mask-face-mask-chicago?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=sr_gallery-246-24&frs=1
https://www.etsy.com/listing/795112076/protective-face-maskwine-is-cheaper-than?ga_or

In [98]:
len(Pdt.keys())

10332

In [100]:
Pdt

{'259063995': {'price': 15.99,
  'in_stock': 9999,
  'title': 'PersonalizedCustomCottonMonogrammedFaceMask,Reusable,DoubleLayer,ProtectiveFaceMask,ClothFacemask,ProtectiveFaceCovering',
  'Date': '2020-06-05'},
 '235071177': {'price': 8,
  'in_stock': 29,
  'title': 'ArtBandanaFaceMask,IllustratedEyeballBandanaFaceMaskCottonBandana',
  'Date': '2020-06-05'},
 '398089661': {'price': 11.2,
  'in_stock': 9999,
  'title': 'BlackFaceMaskWashableReusableElastic',
  'Date': '2020-06-05'},
 '255138068': {'price': 9.99,
  'in_stock': 351,
  'title': 'UnisexAdultFaceMaskWithFilterPocket,ProtectionReusableMouthMask3LayersWashable3DFaceMaskVERYSOFT+10Colorsfacecover',
  'Date': '2020-06-05'},
 '523261039': {'price': 13.87,
  'in_stock': 9999,
  'title': '10PersonalisedFaceMasksPartyPhotoDIYSets-PerfectforBirthdays,HenNights,StagPartiesDoityourselfMustbecutupondelivery',
  'Date': '2020-06-05'},
 '525551721': {'price': 7,
  'in_stock': 295,
  'title': 'FaceCoveringCustomMasks-Samantha?',
  'Date': 

In [102]:
with open('mask_product.json', 'w') as f:
    json.dump(Pdt, f)

## Method II Read csv file

In [39]:

tmp = pd.read_json('Mask_product.json').T
tmp['id'] = tmp.index
tmp.reset_index(drop = True,inplace=True)
tmp.head()

,Date,in_stock,price,title,id
0,2020-06-05,9999,15.99,"PersonalizedCustomCottonMonogrammedFaceMask,Re...",259063995
1,2020-06-05,29,8,"ArtBandanaFaceMask,IllustratedEyeballBandanaFa...",235071177
2,2020-06-05,9999,11.2,BlackFaceMaskWashableReusableElastic,398089661
3,2020-06-05,351,9.99,"UnisexAdultFaceMaskWithFilterPocket,Protection...",255138068
4,2020-06-05,9999,13.87,10PersonalisedFaceMasksPartyPhotoDIYSets-Perfe...,523261039


In [49]:
tmp.size

51660

In [51]:
d1.merge?